In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import awkward as ak
import uproot
import hist
import numpy as np
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea.processor import dict_accumulator, list_accumulator

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5
import itertools
import json

# for making fancy 2d histograms
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils import (plot_1d_trigger_efficiencies, 
                plot_1d_trigger_efficiencies_improvement, 
                plot_fancy_2d_trigger_efficiencies, 
                compare_trigger_efficiencies, 
                plot_cutflow,
                plot_1d_trigger_soup_compact,
                plot_1d_trigger_soup_cms)
from trig_eff_process import (create_baseline_selection_mask, 
                            TriggerEfficiencyProcessor, 
                            TriggerEfficiencyImprovementProcessor, 
                            VBFBaselineCutFlowProcessor, 
                            GeneralBaselineCutFlowProcessor,
                            TriggerSoupProcessor)

In [3]:
for dir in ['2022EE', '2023', '2023BPix']:
    print(f'Started processing {dir}')
    with open(f'/srv/{dir}/{dir}_VBF.json', 'r') as file:
        data = json.load(file)
    samples = data['VBF_Hto2B']
    sample = samples[0]
    events = NanoEventsFactory.from_root(sample, schemaclass=NanoAODSchema).events()
    with open(f'{dir}_HLT.txt', 'w') as f:
        f.write(str(events.HLT.fields))
    print(f'Finished processing {dir}')


Started processing 2022EE
Finished processing 2022EE
Started processing 2023
Finished processing 2023
Started processing 2023BPix
Finished processing 2023BPix


In [4]:
# 2022
init_soup = ['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
triggers_to_add = ['QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1', 'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65']
# 2022EE
init_soup = ['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
triggers_to_add = ['QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1', 'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65']
# 2023
# 'AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet275_SoftDropMass40_PNetBB0p10', 
init_soup = ['AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet230_SoftDropMass40']
# 'PFHT280_QuadPFJet30_PNet2BTagMean0p55', 'PFHT280_QuadPFJet30_PNet2BTagMean0p60', 'PFHT280_QuadPFJet35_PNet2BTagMean0p60', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70',
triggers_to_add = [ 'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']
# 2023BPix
# 'AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet275_SoftDropMass40_PNetBB0p10', 
init_soup = ['AK8PFJet275_SoftDropMass40_PNetBB0p06', 'AK8PFJet230_SoftDropMass40']
# 'PFHT280_QuadPFJet30_PNet2BTagMean0p55', 'PFHT280_QuadPFJet30_PNet2BTagMean0p60', 'PFHT280_QuadPFJet35_PNet2BTagMean0p60', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70',
triggers_to_add = [ 'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']


In [6]:
trigger_dict = {
    '2022': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1',
            'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65'
        ]
    },
    '2022EE': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepCSV_1p3_7p7_VBF1',
            'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65'
        ]
    },
    '2023': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PNetBB0p06',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
            'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70'
        ]
    },
    '2023BPix': {
        'initial_soup': [
            'AK8PFJet275_SoftDropMass40_PNetBB0p06',
            'AK8PFJet230_SoftDropMass40'
        ],
        'triggers_to_add': [
            'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
            'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70'
        ]
    }
}

print(trigger_dict['2022EE']['initial_soup'])
print(trigger_dict['2023']['triggers_to_add'])


['AK8PFJet275_SoftDropMass40_PFAK8ParticleNetBB0p35', 'AK8PFJet230_SoftDropMass40']
['QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1', 'PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70']
